Poniższy dokument stanowi opracowanie części projektu dotyczącej analizy modeli regresji. 

In [1]:
import datetime, warnings
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import Lasso,LinearRegression,Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor,BaggingRegressor
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
from sklearn.model_selection import GridSearchCV

# Wczytanie danych 

Wczytano dane po preprocessingu wykonanym podczas eksploracyjnej analizy danych.

In [ ]:
flights=pd.read_csv("flights_explored.csv")

# Przygotowanie danych do modeli

Przygotowanie wartości kategorycznych

In [ ]:
le = LabelEncoder()

In [ ]:
flights['AIRLINE']= le.fit_transform(flights['AIRLINE'])
flights['ORIGIN_AIRPORT'] = le.fit_transform(flights['ORIGIN_AIRPORT'])
flights['DESTINATION_AIRPORT'] = le.fit_transform(flights['DESTINATION_AIRPORT'])
flights = flights.drop(['SCHEDULED_DEPARTURE','SCHEDULED_ARRIVAL'], axis = 1)

## Dane do trenowania modeli bez uwzględniania DEPARTURE_DELAY

Zrezygnowano jeszcze z kilku cech, w tym z niewygodnych danych opisujących czasy (wylotu/przylotu).

In [ ]:
X = flights.drop(['ARRIVAL_DELAY','TAIL_NUMBER','DEPARTURE_DELAY','DEPARTURE_TIME','ARRIVAL_TIME'],axis = 1)
X.info()

Cechą wynikową jest opóźnienie po przylocie.

In [ ]:
y = flights['ARRIVAL_DELAY']
y.head()

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state = 2)

In [ ]:
sc1=StandardScaler()
X_train_sc=sc1.fit_transform(X_train)
X_test_sc=sc1.transform(X_test)

## Dane do końcowego testu wybranego modelu uwzględniając DEPARTURE_DELAY

In [ ]:
'''
flights=pd.read_csv("flights_explored.csv")
flights['AIRLINE']= le.fit_transform(flights['AIRLINE'])
flights['ORIGIN_AIRPORT'] = le.fit_transform(flights['ORIGIN_AIRPORT'])
flights['DESTINATION_AIRPORT'] = le.fit_transform(flights['DESTINATION_AIRPORT'])
flights = flights.drop(['SCHEDULED_DEPARTURE','SCHEDULED_ARRIVAL'], axis = 1)
'''

In [ ]:
X_d = flights.drop(['ARRIVAL_DELAY','TAIL_NUMBER','DEPARTURE_TIME','ARRIVAL_TIME'],axis = 1)
y_d = flights['ARRIVAL_DELAY']

In [ ]:
X_train_d,X_test_d,y_train_d,y_test_d = train_test_split(X_d,y_d,test_size=0.3,random_state = 2)

In [ ]:
sc1=StandardScaler()
X_train_sc_d=sc1.fit_transform(X_train_d)
X_test_sc_d=sc1.transform(X_test_d)

# Regularyzacja -- dobór parametru dla modelu Lasso i Ridge regresji

Wykorzystaliśmy parametr cv=2 GridSearcgCV, aby na niespacjalnie wydolnym sprzęcie w sensownym czasie rozprawić się z danymi.

In [ ]:
def regularyzacja(model,param):
    model_regressor=GridSearchCV(Lasso(),param,scoring='neg_mean_squared_error',cv=2,verbose=10)
    model_regressor.fit(X_train_sc,y_train)
    print(model_regressor.best_params_)
    print(model_regressor.best_score_)

In [ ]:
param={'alpha':[1e-20,1e-8,1e-3,10]}
regularyzacja(Ridge(),param)

In [ ]:
regularyzacja(Lasso(),param)

W naszym przypadku zauważyliśmy, że im mniejszy hiperparametr, tym lepsze osiągane wyniki (aczkolwiek nadal mało satysfakcjonujące)

# Przygotowanie modeli

In [ ]:
Las = Lasso(alpha=1e-20)
LinR = LinearRegression()
Rid = Ridge(alpha=1e-20)
Boost_Lin = AdaBoostRegressor(base_estimator=LinR,random_state=2)
Boost_las = AdaBoostRegressor(base_estimator=Las,random_state=2)
Boost_rid = AdaBoostRegressor(base_estimator=Rid,random_state=2)
Bg_Lin = BaggingRegressor(base_estimator=LinR,random_state=2)
Bg_las = BaggingRegressor(base_estimator=Las,random_state=2)
Bg_rid = BaggingRegressor(base_estimator=Rid,random_state=2)

# Podstawowe regressory

In [ ]:
def errors_for_regressor(X_train,X_test,y_train,models,names):
    for model, name in zip(models,names):
        model1 = model.fit(X_train,y_train)
        Y_predict=model1.predict(X_test)
        print(name)
        print('Mean Absolute Error:', mean_absolute_error(y_test, Y_predict))  
        print('Mean Squared Error:', mean_squared_error(y_test, Y_predict))  
        print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test, Y_predict)))
        print('R2 : ',r2_score(y_test, Y_predict))
        print()
        print(name)
        plt.scatter(y_test, Y_predict)
        plt.title("Model Analysis")
        plt.xlabel("Real Value")
        plt.ylabel("Prediction")
        plt.show()

In [ ]:
models=[Las,LinR,Rid]
names=['Lasso','Linear','Ridge']

Analiza błędów dla modeli uwzględniających regularyzację. W poniższych przykładach nie wzięliśmy pod uwagę cechy DEPARTURE_DELAY.

Do porównania jakości modeli predykcyjnych zastosowaliśmy następujące metryki:
- MAE
- MSE
- RMSE
- R2

In [ ]:
errors_for_regressor(X_train_sc,X_test_sc,y_train,models,names)

W poniższych przykładach wzięliśmy pod uwagę cechę DEPARTURE_DELAY.

In [ ]:
errors_for_regressor(X_train_sc_d,X_test_sc_d,y_train_d,models,names)

Czego się można było spodziewać, przy użyciu mocno skorelowanej z wyjściem cechy DEPARTURE_DELAY, dokładność modelu drastycznie poprawiła się.

# Boosting & bagging

Spróbowaliśmy również przeprowadzić boosting i bagging podstawowych modeli regresyjnych, ponownie porównaliśmy wyniki otrzymane przy uwzględnieniu mocno skorelowanej cechy DEPARTURE_DELAY oraz bez niej.

## Boosting

Spróbowaliśmy skorzystać z AdaBoost

In [ ]:
errors_for_regressor(X_train_sc,X_test_sc,y_train,[Boost_Lin,Boost_las,Boost_rid],['Boosted Linear','Boosted Lasso','Boosted Ridge'])

In [ ]:
errors_for_regressor(X_train_sc_d,X_test_sc_d,y_train_d,[Boost_Lin,Boost_las,Boost_rid],['Boosted Linear','Boosted Lasso','Boosted Ridge'])

Widoczne jest pogorszenie wyników regresji. Próba zastosowania AdaBoost zakończyła się porażką.

# Bagging

In [ ]:
errors_for_regressor(X_train_sc,X_test_sc,y_train,[Bg_Lin,Bg_las,Bg_rid],['Boosted Linear','Boosted Lasso','Boosted Ridge'])

In [ ]:
errors_for_regressor(X_train_sc_d,X_test_sc_d,y_train_d,[Bg_Lin,Bg_las,Bg_rid],['Bagged Linear','Bagged Lasso','Bagged Ridge'])

# Wnioski 

* Do problemu stworzenia modelu regresyjnego podeszliśmy z założeniem wykorzystania koncepcji poznanych na zajęciach laboratoryjnych i wykładzie. Tymi technikami są regresja, regularyzacja, walidacja krzyżowa, boosting oraz bagging, metryki oceny jakości modelu. W zależności od tego czy korzystaliśmy w naszych predykcjach z jednej bardzo mocno skorelowanej z wyjściem cechy, można stwierdzić że odnieśliśmy spektakularny sukces czy ponieśliśmy sromotną porażkę. Na pewno naszym osobistym sukcesem jest fakt, że przećwiczyliśmy wiele technik poznanych na zajęciach.
* Zastosowanie regularyzacji znacznie poprawiło wyniki w niektórych analizowanych przypadkach (np. zastosowania Lasso Regression).
* Zastosowane metod boostingu oraz baggingu w przypadku naszych modeli bez zastosowania mocno skorelowanej cechy nie było pomocne. W przypadku korzystania z mocno skorelowanej cechy, zazobserowaliśmy znaczną poprawę metryk oceny jakości modelu.
* Wielokrotnie prace trwały długo ze względu na słabą wydajność prywatnych maszyn. Pomijając czas pracy maszyn, najdłużej w całym projekcie zastanawialiśmy się nad doborem cech i oczyszczeniem danych.
* Porównując wyniki użytych metryk oceny skuteczności modelu możemy stwierdzić, że w obu przypakach z uwzlędnianiem lub nie, cechę DEPARTURE_DELAY możemy stwierdzić że do postowionej hipotezy najbardziej się nadają podstawowe modele regresyjne (czyli Lasso, Ridge z regularyzacją oraz Liniowa regresja). 

Wnioski do części projektu będącej analizą eksplorayjną zawarto w notatniku z tą analizą.